# Following the Preprint to Published Path

The goal of this notebook is to map preprint dois to published dois and published dois to Pubmed Central articles.

In [1]:
from habanero import Crossref
import pandas as pd
from ratelimit import limits, sleep_and_retry
import tqdm
from urllib.error import HTTPError

In [2]:
preprints_df = pd.read_csv("../exploratory_data_analysis/output/biorxiv_article_metadata.tsv", sep="\t")
preprints_df.head()

,author_type,heading,category,document,doi
0,regular article,new results,genetics,440735_v1.xml,10.1101/440735
1,regular article,new results,systems biology,775270_v1.xml,10.1101/775270
2,regular article,new results,genetics,242404_v1.xml,10.1101/242404
3,regular article,new results,neuroscience,872994_v1.xml,10.1101/2019.12.11.872994
4,regular article,new results,developmental biology,080853_v2.xml,10.1101/080853


In [3]:
dois = (
    preprints_df
    .doi
    .unique()
)
print(len(dois))

71118


In [4]:
cf = Crossref(mailto="dnicholson329@gmail.com")

In [5]:
TEN_MINUTES = 600

@sleep_and_retry
@limits(calls=10, period=TEN_MINUTES)
def call_crossref(doi_ids):
    responses = []
    for doi in doi_ids:
        try:
            response = cf.works(ids=doi)
            responses.append(response)
        except:
            responses.append({
                "message":{
                    "relation":{"none":"none"}, 
                    "DOI":doi
                }
            })
        
    return responses

# Map preprint DOIs to Published DOIs

In [8]:
batch_limit = 100
doi_mapper_records = []

for batch in tqdm.tqdm(range(0, len(dois), batch_limit)):
    response = call_crossref(dois[batch:batch+batch_limit])
    doi_mapper_records += [
        {
         "preprint_doi":result['message']['DOI'],
         "published_doi":(
                 result['message']['relation']['is-preprint-of'][0]['id'] 
                 if "is-preprint-of" in result['message']['relation'] 
                 else ""
             )
        }
        for result in response
    ]


100%|██████████| 615/615 [10:10:57<00:00, 59.61s/it]   


In [9]:
published_doi_df = (
    pd.DataFrame.from_records(doi_mapper_records)
    .append(mapped_preprints_df)
)
published_doi_df.to_csv("output/mapped_published_doi.tsv", sep="\t", index=False)

# Map Journal Titles to DOI

In [8]:
published_doi_df = pd.read_csv("output/mapped_published_doi.tsv", sep="\t")

In [11]:
journal_dois = (
    published_doi_df
    .published_doi
    .unique()
    .tolist()
)

In [14]:
batch_limit = 100
journal_mapper_records = []

for batch in tqdm.tqdm(range(0, len(journal_dois), batch_limit)):
    response = call_crossref(journal_dois[batch:batch+batch_limit])
    journal_mapper_records += [
        {
            "published_doi": result['message']['DOI'],
            "journal": (
                result['message']['container-title'][0] 
                if 'container-title' in result['message'] and len(result['message']['container-title']) > 0
                else "fill_me_in"
            ),
        }
        for result in response
    ]

100%|██████████| 30/30 [20:08<00:00, 40.29s/it] 


In [15]:
journal_mapper_df = pd.DataFrame.from_records(journal_mapper_records)
journal_mapper_df.head()

,published_doi,journal
0,10.7554/elife.45779,eLife
1,10.1113/jp278773,The Journal of Physiology
2,10.1364/oe.27.019950,Optics Express
3,10.1364/boe.11.000008,Biomedical Optics Express
4,10.1128/mbio.01516-16,mBio


In [12]:
final_df = (
    preprints_df
    .merge(published_doi_df, left_on="doi", right_on="preprint_doi")
    .merge(journal_mapper_df, on="published_doi", how="left")
    .drop("preprint_doi", axis=1)
)
final_df.head()

,author_type,heading,category,document,doi,published_doi,journal
0,regular article,new results,genetics,440735_v1.xml,10.1101/440735,NaN,NaN
1,regular article,new results,genetics,440735_v3.xml,10.1101/440735,NaN,NaN
2,regular article,new results,genetics,440735_v2.xml,10.1101/440735,NaN,NaN
3,regular article,new results,systems biology,775270_v1.xml,10.1101/775270,NaN,NaN
4,regular article,new results,genetics,242404_v1.xml,10.1101/242404,NaN,NaN


In [13]:
final_df.to_csv("output/mapped_published_doi.tsv", sep="\t", index=False)

# Map Published Articles to PMC

In [6]:
preprint_df = pd.read_csv("output/mapped_published_doi.tsv", sep="\t")
preprint_df.head()

,author_type,heading,category,document,doi,published_doi,journal
0,regular article,new results,genetics,440735_v1.xml,10.1101/440735,NaN,NaN
1,regular article,new results,genetics,440735_v3.xml,10.1101/440735,NaN,NaN
2,regular article,new results,genetics,440735_v2.xml,10.1101/440735,NaN,NaN
3,regular article,new results,systems biology,775270_v1.xml,10.1101/775270,NaN,NaN
4,regular article,new results,genetics,242404_v1.xml,10.1101/242404,NaN,NaN


In [7]:
pmc_df = pd.read_csv(
    "../../pmc/exploratory_data_analysis/output/pubmed_central_journal_paper_map.tsv.xz", 
    sep="\t"
)
pmc_df.head()

,journal,article_type,doi,pmcid
0,Environ_Health,research-article,10.1186/1476-069X-5-22,PMC1552054
1,Environ_Health,research-article,10.1186/1476-069X-4-12,PMC1226148
2,Environ_Health,correction,10.1186/s12940-018-0415-9,PMC6124016
3,Environ_Health,research-article,10.1186/s12940-017-0316-3,PMC5635510
4,Environ_Health,research-article,10.1186/1476-069X-10-46,PMC3125232


In [8]:
final_df = (
    preprint_df
    .merge(
        pmc_df[["doi", "pmcid"]].dropna(), 
        how="left", left_on="published_doi", 
        right_on="doi"
    )
    .drop("doi_y", axis=1)
    .rename(index=str, columns={"doi_x":"doi"})
)
final_df.head()

,author_type,heading,category,document,doi,published_doi,journal,pmcid
0,regular article,new results,genetics,440735_v1.xml,10.1101/440735,NaN,NaN,NaN
1,regular article,new results,genetics,440735_v3.xml,10.1101/440735,NaN,NaN,NaN
2,regular article,new results,genetics,440735_v2.xml,10.1101/440735,NaN,NaN,NaN
3,regular article,new results,systems biology,775270_v1.xml,10.1101/775270,NaN,NaN,NaN
4,regular article,new results,genetics,242404_v1.xml,10.1101/242404,NaN,NaN,NaN


In [9]:
final_df.to_csv("output/mapped_published_doi.tsv", sep="\t", index=False)